In [ ]:
import numpy as np
import MCTS
import naiveRule
import matplotlib.pyplot as plt
import RNG
import struct
%config InlineBackend.figure_formats = ['svg']

In [ ]:
%%capture
MCTS.loadEngine(1,"./weights/RNG64.tf")
MCTS.loadEngine(2,"./RNG64.tf")

In [ ]:
fn="games/gm5.x"
sz=os.stat(fn).st_size//4
fp=open(fn,"rb")
x_tr=struct.unpack('f'*sz, fp.read(4*sz))
x_tr=np.reshape(x_tr,[sz//450,15,15,2])
fp.close()
fn="games/gm5.y"
sz=os.stat(fn).st_size//4
fp=open(fn,"rb")
y_tr=struct.unpack('f'*sz, fp.read(4*sz))
y_tr=np.reshape(y_tr,[sz//226,226])
fp.close()
print(len(x_tr),len(y_tr))
print(np.average(y_tr[:,-1]))
idc=np.random.permutation(len(x_tr))
print(MCTS.a0eng .a0_eng.evaluate(x_tr[idc],y_tr[idc],return_dict=True))
print(MCTS.a0eng2.a0_eng.evaluate(x_tr[idc],y_tr[idc],return_dict=True))

In [ ]:
def flip_x(ps):
    return (14-ps//15)*15+ps%15
def flip_y(ps):
    return (ps//15)*15+(14-ps%15)
def trs_xy(ps):
    return (ps%15)*15+ps//15
idxes=[# here it should be the inverse operation!!!
    np.array([(ps if ps<225 else ps) for ps in range(226)]),
    np.array([(flip_x(ps) if ps<225 else ps) for ps in range(226)]),
    np.array([(flip_y(ps) if ps<225 else ps) for ps in range(226)]),
    np.array([(flip_x(flip_y(ps)) if ps<225 else ps) for ps in range(226)]),
    np.array([(trs_xy(ps) if ps<225 else ps) for ps in range(226)]),
    np.array([(trs_xy(flip_y(ps)) if ps<225 else ps) for ps in range(226)]),
    np.array([(trs_xy(flip_x(ps)) if ps<225 else ps) for ps in range(226)]),
    np.array([(trs_xy(flip_x(flip_y(ps))) if ps<225 else ps) for ps in range(226)]),
]
mxfns=[
    lambda x: x,
    lambda x: np.flip(x,0),
    lambda x: np.flip(x,1),
    lambda x: np.flip(x,(0,1)),
    lambda x: np.transpose(x,(1,0,2)),
    lambda x: np.transpose(np.flip(x,0),(1,0,2)),
    lambda x: np.transpose(np.flip(x,1),(1,0,2)),
    lambda x: np.transpose(np.flip(x,(0,1)),(1,0,2)),
]
def data_augmentator(datx,daty,rnd):
    if daty is None:
        return mxfns[rnd](datx)
    return mxfns[rnd](datx),daty[idxes[rnd]]

In [ ]:
x_tra,y_tra=x_tr.copy(),y_tr.copy()
for kk in range(len(x_tra)):
    x_tra[kk],y_tra[kk]=data_augmentator(x_tra[kk],y_tra[kk],np.random.randint(8))

In [ ]:
print(MCTS.a0eng .a0_eng.evaluate(x_tra,y_tra,return_dict=True))
print(MCTS.a0eng2.a0_eng.evaluate(x_tra,y_tra,return_dict=True))

In [ ]:
npz_t=np.load("./games/dat_train0.npz")
x_tr=npz_t['arr_0']
y_tr=npz_t['arr_1']
print(len(y_tr))
print(np.average(y_tr[:,-1]))
idc=np.random.permutation(len(x_tr))
print(MCTS.a0eng .a0_eng.evaluate(x_tr[idc],y_tr[idc],return_dict=True))
print(MCTS.a0eng2.a0_eng.evaluate(x_tr[idc],y_tr[idc],return_dict=True))

In [ ]:
i=0
def maxprb(dat1,ivt):
    yt=dat1**ivt
    return np.sort(yt/np.sum(yt))[-2:]
invT=np.linspace(.5,2.5,20)
def drwPrb(dat1,dat2=None):
    mxp=np.transpose(np.array([maxprb(dat1,it) for it in invT]))
    plt.figure(figsize=(2.8,1.4))
    for lns in mxp:
        plt.plot(invT,lns,color="black")
    if dat2 is not None:
        mxp=np.transpose(np.array([maxprb(dat2,it) for it in invT]))
        for lns in mxp:
            plt.plot(invT,lns,color="red")
    plt.grid()

In [ ]:
nnrst=MCTS.a0eng.a0_eng(np.array([x_tr[i]]))[0].numpy()
nnrsta=MCTS.a0eng.a0_eng(np.array([x_tra[i]]))[0].numpy()
MCTS.showHeatMap(x_tr[i],y_tr[i],nnrst)
MCTS.showHeatMap(x_tra[i],y_tra[i],nnrsta)
# drwPrb(y_tr[i,:-1],nnrst[:-1])
i+=1

In [ ]:
i-=2

In [ ]:
xt=np.zeros([15,15,2])
xt[5,7,0]=1
xt[6,8,1]=1
xt[6,6,0]=1
xt[4,8,1]=1
xt[7,7,0]=1
xt[5,8,1]=1
xt[3,8,0]=1
xt[4,7,1]=1
xt[6,9,0]=1
xt[4,9,1]=1
xt[4,10,0]=1
xt[4,5,1]=1
xt[4,6,0]=1
xt[6,7,1]=1
xt[7,6,0]=1
xt=np.flip(xt,-1)

In [ ]:
xt=np.zeros([15,15,2])
xt[7,7,0]=1
xt[6,7,1]=1
xt[9,9,0]=1
xt=np.flip(xt,-1)

In [ ]:
xta=data_augmentator(xt,None,1)
MCTS.showHeatMap(xta,MCTS.a0eng.a0_eng(np.array([xta]))[0].numpy(),MCTS.a0eng2.a0_eng(np.array([xta]))[0].numpy())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_formats = ['svg']
fp=open("trLog64.log","r")
lsv=[]
pv=[]
vv=[]
lst=[]
pt=[]
vt=[]
rr=[]
dbl=[]
while(True):
    ss=fp.readline()
    if(ss==""):
        break
    es=ss.split()
    if("training" in es):
        vv+=[float(es[-1])]
        pv+=[float(es[-3])]
        lsv+=[float(es[-5])]
        vt+=[float(es[-7])]
        pt+=[float(es[-9])]
        lst+=[float(es[-11])]
    elif("dbls" in es):
        rr+=[float(es[4])]
        dbl+=[int(es[7])]
fp.close()

elo=[]
elot=[]
elo_teacher=0.0
def rrt2elo(rat,el_tr):
    rrat=min(max(0.01,rat),0.99)
    return el_tr-np.log10((1-rrat)/rrat)*400
for i in range(len(dbl)):
    elo+=[rrt2elo(rr[i]/64,elo_teacher)]
    elot+=[elo_teacher]
    if(rr[i]-dbl[i]/2>32):
        elo_teacher=elo[-1]

_, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2,2,figsize=(7/.618,7))
ax1.plot(lst)
ax1.plot(lsv)
ax1.grid(which='both')
ax1.set_title("loss")
ax1.set_ylim([0,np.min(lsv)*2])
ax2.plot(pt)
ax2.plot(pv)
ax2.grid(which='both')
ax2.set_title("p accuracy")
ax2.set_ylim([.0,(1+max(np.max(pv),np.max(pt)))/2])
ax3.plot(vt)
ax3.plot(vv)
ax3.grid(which='both')
ax3.set_title("v accuracy")
ax3.set_ylim([.45,(1+max(np.max(vv),np.max(vt)))/2])
ax4.plot(elot)
ax4.plot(elo)
ax4.grid(which='both')
ax4.set_title("elo rating")
ax4.set_ylim([max(elo[-1],elot[-1])*.5,max(elo[-1],elot[-1])*1.05])
plt.savefig("training.pdf")

In [ ]:
import time
import datetime
fp=open("trLog64.log","r")
fp1=open("trLog128.log","r")
fp2=open("trLog20.log","r")
lsv,pv,vv,lst,pt,vt=[],[],[],[],[],[]
lsv1,pv1,vv1,lst1,pt1,vt1=[],[],[],[],[],[]
lsv2,pv2,vv2,lst2,pt2,vt2=[],[],[],[],[],[]
while(True):
    ss1=fp1.readline()
    ss2=fp2.readline()
    if(ss1==""):
        break
    es1=ss1.split()
    es2=ss2.split()
    if("training" in es1):
        t1=time.mktime(datetime.datetime.strptime(es1[0]+" "+es1[1], "%m/%d/%Y %H:%M:%S").timetuple())
        vv1+=[float(es1[-1])]
        pv1+=[float(es1[-3])]
        lsv1+=[float(es1[-5])]
        vt1+=[float(es1[-7])]
        pt1+=[float(es1[-9])]
        lst1+=[float(es1[-11])]
        vv2+=[float(es2[-1])]
        pv2+=[float(es2[-3])]
        lsv2+=[float(es2[-5])]
        vt2+=[float(es2[-7])]
        pt2+=[float(es2[-9])]
        lst2+=[float(es2[-11])]
        while(True):
            es=fp.readline().split()
            if("training" in es):
                ess=es
            try:
                t2=time.mktime(datetime.datetime.strptime(es[0]+" "+es[1], "%m/%d/%Y %H:%M:%S").timetuple())
            except:
                t2=0
            if(t1<t2):
                vv+=[float(ess[-1])]
                pv+=[float(ess[-3])]
                lsv+=[float(ess[-5])]
                vt+=[float(ess[-7])]
                pt+=[float(ess[-9])]
                lst+=[float(ess[-11])]
                break

fp.close()
fp1.close()
fp2.close()

_, ((ax1,ax2,ax3)) = plt.subplots(1,3,figsize=(12,4))
lb=1
for ln in [lst,lsv,lst1,lsv1,lst2,lsv2]:
    ax1.plot(ln,"o-",label=str(lb))
    lb+=1
ax1.grid(which='both')
ax1.set_title("loss")
ax1.legend()
for ln in [pt,pv,pt1,pv1,pt2,pv2]:
    ax2.plot(ln,"o-")
ax2.grid(which='both')
ax2.set_title("p accuracy")
for ln in [vt,vv,vt1,vv1,vt2,vv2]:
    ax3.plot(ln,"o-")
ax3.grid(which='both')
ax3.set_title("v accuracy")

In [ ]:
if(os.path.isfile("./games/dat_trprev.npz")):
    npz_t=np.load("./games/dat_trprev.npz")
    x_tr=npz_t['arr_0']
    y_tr=npz_t['arr_1']
    print("load cached data: %d"%(len(y_tr)))
    npz_t=np.load("./games/dat_train1.npz")
    x_tr=np.concatenate((x_tr,npz_t['arr_0']))
    y_tr=np.concatenate((y_tr,npz_t['arr_1']))
else:
    npz_t=np.load("./games/dat_train1.npz")
    x_tr=npz_t['arr_0']
    y_tr=npz_t['arr_1']

npz_v=np.load("./games/dat_vlidn1.npz")
x_vl=npz_v['arr_0']
y_vl=npz_v['arr_1']

for ii in range(2,33):
    nf="./games/dat_train"+str(ii)+".npz"
    if(os.path.exists(nf)):
        npz_t=np.load(nf)
        x_tr=np.concatenate((x_tr,npz_t['arr_0']))
        y_tr=np.concatenate((y_tr,npz_t['arr_1']))
    nf="./games/dat_vlidn"+str(ii)+".npz"
    if(os.path.exists(nf)):
        npz_v=np.load(nf)
        x_vl=np.concatenate((x_vl,npz_v['arr_0']))
        y_vl=np.concatenate((y_vl,npz_v['arr_1']))

In [ ]:
bz=256
sg=np.std(x_tr,axis=(0,1,2))
mu=np.mean(x_tr,axis=(0,1,2))
for i in range(25):
    bsg=np.std(x_tr[bz*i:bz*(i+1)],axis=(0,1,2))
    bmu=np.mean(x_tr[bz*i:bz*(i+1)],axis=(0,1,2))
    print(bsg/sg,(bmu-mu)/sg)
    if(bz*(i+1)>len(y_tr)):
        break

In [ ]:
%%capture
nw=0
wts=[]
while(os.path.isfile("./weights/RNG64_%d.tf.index"%(nw+1))):
    wts.append(RNG.A0_ENG(64,"./weights/RNG64_%d.tf"%(nw+1)).a0_eng.get_weights())
    nw+=1
nw+=1
wts.append(RNG.A0_ENG(64,"./RNG64.tf").a0_eng.get_weights())

In [ ]:
for ele in range(50):
    idx=tuple(np.random.randint(dd) for dd in wts[0][ele].shape)
    plt.plot([wts[i][ele][idx] for i in range(nw)])

In [ ]:
for ele in range(-50,0):
    plt.plot([np.sqrt(np.mean(wts[i][ele]**2)) for i in range(nw)])

In [ ]:
def drawEval(no,ax):
    fp=open("./eval/gm%d.txt"%(no),"r")
    # fp.readline()
    mv=[]
    op=[]
    while(True):
        tx=fp.readline()
        if(tx==""):
            break
        ss=tx.split()
        if(len(ss)==7):
            mv.append([int(ss[0]),int(ss[1]),float(ss[4])])
        if("b" in ss):
            op.append([int(ss[2]),int(ss[3])])
    fp.close()
    spl=1
    while(mv[spl][0]>mv[spl-1][0]):
        spl+=1
    for i in range(len(mv)):
        mv[i][2]=mv[i][2] if mv[i][1] else 1.0-mv[i][2]
    eng1=np.transpose([[mv[i][0],mv[i][2]] for i in range(len(mv)) if mv[i][1]==1])
    eng2=np.transpose([[mv[i][0],mv[i][2]] for i in range(len(mv)) if mv[i][1]==0])
    gm1=np.transpose(mv[:spl])
    gm2=np.transpose(mv[spl:])
    ax.plot(gm1[0],gm1[2]*2-1,"cyan")
    ax.plot(gm2[0],gm2[2]*2-1,"magenta")
    ax.plot(eng1[0],eng1[1]*2-1,"ro",label="New",markersize=4)
    ax.plot(eng2[0],eng2[1]*2-1,"bv",label="Old",markersize=4)
    ax.grid()
    ax.set_ylim(-1.1,1.1)
    ax.set_title(str(op[:len(op)//2]))
    # ax.legend()

In [ ]:
_,axs=plt.subplots(8,4,figsize=(14,28))
for k in range(8):
    for l in range(4):
        drawEval(k*4+l+1,axs[k][l])
plt.savefig("evals.pdf")